# TCR Clonality Analysis
**Author: Juan Sebastian Diaz Boada**<br>
*juan.sebastian.diaz.boada@ki.se*<br>
28/12/2022<br>
Environment: `TCR_python`<br>

In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..', 'bin'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from data_functions import read_dataframe, group_with_freq
from data_functions import generate_clone_sets, concat_seqs_in_set

### Settings

In [4]:
pd.set_option('display.max_columns',None)

In [5]:
in_dir = '../data/05_SS3_collected_TCRs/'

# 1. Merge

In [6]:
DF = pd.DataFrame()
for plate in os.listdir(in_dir):
    if plate != '.gitkeep':
        for file in os.listdir(os.path.join(in_dir,plate)):
            df = read_dataframe(os.path.join(in_dir,plate,file))
            df.insert(0,'Plate',plate)
            DF = pd.concat([DF,df])

In [8]:
del(df)

In [7]:
DF

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J
Plate231_PB_K23,SS3_21_231-1,True,170.7400,False,True,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,531.678,False,True,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1075.3400,False,True,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,899.741,False,True,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01
Plate231_MUSL_C14,SS3_21_231-1,True,569.4110,False,True,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,789.405,False,True,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,SS3_21_231-1,True,47.6966,False,True,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1168.820,False,True,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_P19,SS3_21_231-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_O13,SS3_21_231-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_M20,SS3_21_231-3,True,2100.4000,False,True,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4504.390,False,True,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_J13,SS3_21_231-3,True,255.3900,False,True,TRAV4_GGTGAAAGGAACCA_TRAJ23,CTCGTGGGTGAAAGGAACCAGGGAG

# 2. Data treating

In [34]:
df = DF.copy()

In [35]:
df.columns

Index(['Plate', 'A_1_productive', 'A_1_TPM', 'A_1_stop_codon', 'A_1_in_frame',
       'A_1_ID', 'A_1_CDR3nt', 'A_1_CDR3aa', 'A_1_V', 'A_1_J',
       'A_2_productive', 'A_2_TPM', 'A_2_stop_codon', 'A_2_in_frame', 'A_2_ID',
       'A_2_CDR3nt', 'A_2_CDR3aa', 'A_2_V', 'A_2_J', 'B_1_productive',
       'B_1_TPM', 'B_1_stop_codon', 'B_1_in_frame', 'B_1_ID', 'B_1_CDR3nt',
       'B_1_CDR3aa', 'B_1_V', 'B_1_J', 'B_1_D', 'B_2_productive', 'B_2_TPM',
       'B_2_stop_codon', 'B_2_in_frame', 'B_2_ID', 'B_2_CDR3nt', 'B_2_CDR3aa',
       'B_2_V', 'B_2_J', 'B_2_D', 'G_1_productive', 'G_1_TPM',
       'G_1_stop_codon', 'G_1_in_frame', 'G_1_ID', 'G_1_CDR3nt', 'G_1_CDR3aa',
       'G_1_V', 'G_1_J', 'D_1_productive', 'D_1_TPM', 'D_1_stop_codon',
       'D_1_in_frame', 'D_1_ID', 'D_1_CDR3nt', 'D_1_CDR3aa', 'D_1_V', 'D_1_J',
       'D_1_D', 'G_2_productive', 'G_2_TPM', 'G_2_stop_codon', 'G_2_in_frame',
       'G_2_ID', 'G_2_CDR3nt', 'G_2_CDR3aa', 'G_2_V', 'G_2_J'],
      dtype='object')

In [36]:
# Replace Nans for zero in productive columns
cols = df.columns[df.columns.str.endswith('productive')|\
                    df.columns.str.endswith('stop_codon')|\
                    df.columns.str.endswith('in_frame')]
for i in cols:
    df.loc[:,i] = df.loc[:,i].fillna(0).astype(int)
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J
Plate231_PB_K23,SS3_21_231-1,1,170.7400,0,1,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,531.678,0,1,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1075.3400,0,1,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,899.741,0,1,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01
Plate231_MUSL_C14,SS3_21_231-1,1,569.4110,0,1,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,789.405,0,1,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,SS3_21_231-1,1,47.6966,0,1,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1168.820,0,1,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
Plate231_PB_P19,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
Plate231_PB_O13,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
Plate231_PB_M20,SS3_21_231-3,1,2100.4000,0,1,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,4504.390,0,1,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN
Plate231_PB_J13,SS3_21_231-3,1,255.3900,0,1,TRAV4_GGTGAAAGGAACCA_TRAJ23,CTCGTGGGTGAAAGGAACCAGGGAGGAAAGCTTATC,LVGERNQGGKLI,TRAV4*01,TRAJ23*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1266.860,0,1,TRBV3-1_CAGCCGGGACAGCAA_TRBJ1-5,GCCAGCAGCCGGGACAGCAATCAGCCCCAGCAT,ASSRDSNQPQH,TRBV3-1*01,TRBJ1-5*01,"TRBD1*01,TRBD2*01,TRBD2*02",0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,Na

In [37]:
# Fill missing data
loci = ['A_1','A_2','B_1','B_2','G_1','G_2','D_1','D_2']
for l in loci:
    if not np.any(df.columns.str.contains(l)):
        df.insert(len(df.columns),l+'_productive',0)
        df.insert(len(df.columns),l+'_TPM',np.nan)
        df.insert(len(df.columns),l+'_stop_codon',0)
        df.insert(len(df.columns),l+'_in_frame',0)
        df.insert(len(df.columns),l+'_ID',np.nan)
        df.insert(len(df.columns),l+'_CDR3nt',np.nan)
        df.insert(len(df.columns),l+'_CDR3aa',np.nan)
        df.insert(len(df.columns),l+'_V',np.nan)
        df.insert(len(df.columns),l+'_J',np.nan)
        if l in ['B_1','B_2','D_1','D_2']:
            df.insert(len(df.columns),l+'_D',np.nan)

In [38]:
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J,D_2_productive,D_2_TPM,D_2_stop_codon,D_2_in_frame,D_2_ID,D_2_CDR3nt,D_2_CDR3aa,D_2_V,D_2_J,D_2_D
Plate231_PB_K23,SS3_21_231-1,1,170.7400,0,1,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,531.678,0,1,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1075.3400,0,1,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,899.741,0,1,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_C14,SS3_21_231-1,1,569.4110,0,1,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,789.405,0,1,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,SS3_21_231-1,1,47.6966,0,1,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1168.820,0,1,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_P19,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_O13,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_M20,SS3_21_231-3,1,2100.4000,0,1,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,4504.390,0,1,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_J13,SS3_21_231-3,1,255.3900,0,1,TRAV4_GGTGAAAGGAACCA_

In [39]:
# Reorder columns
loci = ['A_1','A_2','B_1','B_2','G_1','G_2','D_1','D_2']
new_cols = list([df.columns[0]])
for l in loci:
    new_cols = new_cols + list(df.columns[df.columns.str.startswith(l)])
df = df[new_cols]
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,D_2_productive,D_2_TPM,D_2_stop_codon,D_2_in_frame,D_2_ID,D_2_CDR3nt,D_2_CDR3aa,D_2_V,D_2_J,D_2_D
Plate231_PB_K23,SS3_21_231-1,1,170.7400,0,1,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,531.678,0,1,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1075.3400,0,1,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",1,899.741,0,1,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_C14,SS3_21_231-1,1,569.4110,0,1,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,789.405,0,1,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,SS3_21_231-1,1,47.6966,0,1,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1168.820,0,1,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_P19,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_O13,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_M20,SS3_21_231-3,1,2100.4000,0,1,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,4504.390,0,1,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
Plate231_PB_J13,SS3_21_231-3,1,255.3900,0,1,TRAV4_GGTGAAAGGAACCA_

# 3. $\alpha\beta$ and $\gamma\delta$ productive columns
Create columns of total $\alpha\beta$ and $\gamma\delta$ productives, summing up all their productive values per cell.

In [40]:
# Productive dataframe
P = df.loc[:,df.columns.str.endswith('productive')] 
P

,A_1_productive,A_2_productive,B_1_productive,B_2_productive,G_1_productive,G_2_productive,D_1_productive,D_2_productive
Plate231_PB_K23,1,0,1,0,1,1,0,0
Plate231_MUSL_C14,1,0,1,0,0,0,0,0
Plate231_MUSL_B19,1,0,1,0,0,0,0,0
Plate231_PB_P19,0,0,0,0,0,0,0,0
Plate231_PB_O13,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,0,0,0,0,0,0,0,0
Plate231_PB_M20,1,0,1,0,0,0,0,0
Plate231_PB_J13,1,0,1,0,0,0,0,0
Plate231_MUSL_B24,0,1,1,0,1,0,0,0


In [41]:
loci = ['A','B','G','D']
for l in loci:
    if l in ['A','G']:
        suffix = '_2_J'
    elif l in ['B','D']:
        suffix = '_2_D'
    idx = int(np.where(df.columns==l+suffix)[0][0])
    df.insert(idx+1,l+'_productive',P.loc[:,P.columns.str.startswith(l)].sum(axis=1))
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,A_productive,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,B_productive,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J,G_productive,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,D_2_productive,D_2_TPM,D_2_stop_codon,D_2_in_frame,D_2_ID,D_2_CDR3nt,D_2_CDR3aa,D_2_V,D_2_J,D_2_D,D_productive
Plate231_PB_K23,SS3_21_231-1,1,170.7400,0,1,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,531.678,0,1,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1075.3400,0,1,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",1,899.741,0,1,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01,2,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
Plate231_MUSL_C14,SS3_21_231-1,1,569.4110,0,1,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,789.405,0,1,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
Plate231_MUSL_B19,SS3_21_231-1,1,47.6966,0,1,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,1168.820,0,1,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
Plate231_PB_P19,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
Plate231_PB_O13,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
Plate231_PB_M20,SS3_21_231-3,1,2100.4000,0,1,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,4504.390,0,1,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,N

In [42]:
idx = int(np.where(df.columns=='B_productive')[0][0])
df.insert(idx+1,'AB_productive',P.iloc[:,:4].sum(axis=1))
df.insert(len(df.columns),'GD_productive',P.iloc[:,4:8].sum(axis=1))
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,A_productive,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,B_productive,AB_productive,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J,G_productive,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,D_2_productive,D_2_TPM,D_2_stop_codon,D_2_in_frame,D_2_ID,D_2_CDR3nt,D_2_CDR3aa,D_2_V,D_2_J,D_2_D,D_productive,GD_productive
Plate231_PB_K23,SS3_21_231-1,1,170.7400,0,1,TRAV19_TGAGTACTGGC_TRAJ38,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,ALSTGNNRKL,TRAV19*01,TRAJ38*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,531.678,0,1,TRBV10-3_CAGTGTTCCAGGGGGGTTGAACTA_TRBJ1-2,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,AISVPGGLNYGYT,TRBV10-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1,1075.3400,0,1,TRGV9_TGGGACAAATCGTCAGGCAG_TRGJ1,GCCTTGTGGGACAAATCGTCA,ALWDKSS,TRGV9*01,"TRGJ1*02,TRGJ2*01",1,899.741,0,1,TRGV9_GGAGGGTCGATATAGT_TRGJP2,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,ALWEGRYSSDWIKT,TRGV9*01,TRGJP2*01,2,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2
Plate231_MUSL_C14,SS3_21_231-1,1,569.4110,0,1,TRAV12-2_TGCCGGGGGGCTAGGAGG_TRAJ9,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,AGGLGGFKTI,TRAV12-2*02,TRAJ9*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,789.405,0,1,TRBV3-1_AGCCAGGGATCGGTCTACG_TRBJ2-7,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,ASSQGSVYEQY,TRBV3-1*01,TRBJ2-7*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
Plate231_MUSL_B19,SS3_21_231-1,1,47.6966,0,1,TRAV9-2_AGTGATCAGG_TRAJ54,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,ALSDQGAQKLV,TRAV9-2*01,TRAJ54*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,1168.820,0,1,TRBV2_TGCCACCGGGAATGA_TRBJ2-1,GCCACCGGGAATGAGCAGTTC,ATGNEQF,TRBV2*01,TRBJ2-1*01,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
Plate231_PB_P19,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
Plate231_PB_O13,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_MUSL_E17,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
Plate231_PB_M20,SS3_21_231-3,1,2100.4000,0,1,TRAV21_TGAGGCCCTTGGGAAATGC_TRAJ39,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,AVRPLGNAGNMLT,TRAV21*01,TRAJ39*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,4504.390,0,1,TRBV6-1_TGCCATTCCAGGTACAGGGAACA_TRBJ1-1,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,AIPGTGNTEAF,TRBV6-1*01,TRBJ1-1*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,NaN,0,0,N

# 4. Masking sequences by productive value

In [43]:
# CDR3 dataframes
CDR3nt = df.loc[:,df.columns.str.endswith('CDR3nt')]
CDR3aa = df.loc[:,df.columns.str.endswith('CDR3aa')]

## AB

In [44]:
AB_CDR3nt = CDR3nt.iloc[:,:4]
AB_CDR3aa = CDR3aa.iloc[:,:4]

In [45]:
AB_mask = P.iloc[:,:4].astype(bool)

In [46]:
# Nucleotide masking
AB_mask.columns = AB_CDR3nt.columns
masked_ABnt = AB_CDR3nt.mask(~AB_mask)
masked_ABnt

,A_1_CDR3nt,A_2_CDR3nt,B_1_CDR3nt,B_2_CDR3nt
Plate231_PB_K23,GCTCTGAGTACTGGCAACAACCGTAAGCTGA,NaN,GCCATCAGTGTTCCAGGGGGGTTGAACTATGGCTACACC,NaN
Plate231_MUSL_C14,GCCGGGGGGCTAGGAGGCTTCAAAACTATC,NaN,GCCAGCAGCCAGGGATCGGTCTACGAGCAGTAC,NaN
Plate231_MUSL_B19,GCTCTGAGTGATCAGGGAGCCCAGAAGCTGGTA,NaN,GCCACCGGGAATGAGCAGTTC,NaN
Plate231_PB_P19,NaN,NaN,NaN,NaN
Plate231_PB_O13,NaN,NaN,NaN,NaN
...,...,...,...,...
Plate231_MUSL_E17,NaN,NaN,NaN,NaN
Plate231_PB_M20,GCTGTGAGGCCCTTGGGAAATGCAGGCAACATGCTCACC,NaN,GCCATTCCAGGTACAGGGAACACTGAAGCTTTC,NaN
Plate231_PB_J13,CTCGTGGGTGAAAGGAACCAGGGAGGAAAGCTTATC,NaN,GCCAGCAGCCGGGACAGCAATCAGCCCCAGCAT,NaN
Plate231_MUSL_B24,NaN,GCTGTGGAGAACAACTCTGGGGCTGGGAGTTACCAACTCACT,GCCAGCAGTGTTGTTGAAGCTTTC,NaN


In [47]:
#  Amino acid masking
AB_mask.columns = AB_CDR3aa.columns
masked_ABaa = AB_CDR3aa.mask(~AB_mask)
masked_ABaa

,A_1_CDR3aa,A_2_CDR3aa,B_1_CDR3aa,B_2_CDR3aa
Plate231_PB_K23,ALSTGNNRKL,NaN,AISVPGGLNYGYT,NaN
Plate231_MUSL_C14,AGGLGGFKTI,NaN,ASSQGSVYEQY,NaN
Plate231_MUSL_B19,ALSDQGAQKLV,NaN,ATGNEQF,NaN
Plate231_PB_P19,NaN,NaN,NaN,NaN
Plate231_PB_O13,NaN,NaN,NaN,NaN
...,...,...,...,...
Plate231_MUSL_E17,NaN,NaN,NaN,NaN
Plate231_PB_M20,AVRPLGNAGNMLT,NaN,AIPGTGNTEAF,NaN
Plate231_PB_J13,LVGERNQGGKLI,NaN,ASSRDSNQPQH,NaN
Plate231_MUSL_B24,NaN,AVENNSGAGSYQLT,ASSVVEAF,NaN


## GD

In [48]:
GD_CDR3nt = CDR3nt.iloc[:,4:8]
GD_CDR3aa = CDR3aa.iloc[:,4:8]

In [49]:
GD_mask = P.iloc[:,4:8].astype(bool)
GD_mask

,G_1_productive,G_2_productive,D_1_productive,D_2_productive
Plate231_PB_K23,True,True,False,False
Plate231_MUSL_C14,False,False,False,False
Plate231_MUSL_B19,False,False,False,False
Plate231_PB_P19,False,False,False,False
Plate231_PB_O13,False,False,False,False
...,...,...,...,...
Plate231_MUSL_E17,False,False,False,False
Plate231_PB_M20,False,False,False,False
Plate231_PB_J13,False,False,False,False
Plate231_MUSL_B24,True,False,False,False


In [50]:
# Nucleotide masking
GD_mask.columns = GD_CDR3nt.columns
masked_GDnt = GD_CDR3nt.mask(~GD_mask)
masked_GDnt

,G_1_CDR3nt,G_2_CDR3nt,D_1_CDR3nt,D_2_CDR3nt
Plate231_PB_K23,GCCTTGTGGGACAAATCGTCA,GCCTTGTGGGAGGGTCGATATAGTAGTGATTGGATCAAGACG,NaN,NaN
Plate231_MUSL_C14,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,NaN,NaN,NaN,NaN
Plate231_PB_P19,NaN,NaN,NaN,NaN
Plate231_PB_O13,NaN,NaN,NaN,NaN
...,...,...,...,...
Plate231_MUSL_E17,NaN,NaN,NaN,NaN
Plate231_PB_M20,NaN,NaN,NaN,NaN
Plate231_PB_J13,NaN,NaN,NaN,NaN
Plate231_MUSL_B24,GGCGTGGAACTCTTT,NaN,NaN,NaN


In [51]:
# Amino acid masking
GD_mask.columns = GD_CDR3aa.columns
masked_GDaa = GD_CDR3aa.mask(~GD_mask)
masked_GDaa

,G_1_CDR3aa,G_2_CDR3aa,D_1_CDR3aa,D_2_CDR3aa
Plate231_PB_K23,ALWDKSS,ALWEGRYSSDWIKT,NaN,NaN
Plate231_MUSL_C14,NaN,NaN,NaN,NaN
Plate231_MUSL_B19,NaN,NaN,NaN,NaN
Plate231_PB_P19,NaN,NaN,NaN,NaN
Plate231_PB_O13,NaN,NaN,NaN,NaN
...,...,...,...,...
Plate231_MUSL_E17,NaN,NaN,NaN,NaN
Plate231_PB_M20,NaN,NaN,NaN,NaN
Plate231_PB_J13,NaN,NaN,NaN,NaN
Plate231_MUSL_B24,GVELF,NaN,NaN,NaN


# 5. Clone definition
Generate clones defined as a unique combination of chains. Clones can be $\alpha\beta$ clones or $\gamma\delta$ clones. Each clone is created by generating a set of all non-NaN sequences coming from any of the alleles of the loci.  

## 5.1 Generate clones

In [52]:
cols = ['A_1_CDR3nt','A_2_CDR3nt','B_1_CDR3nt','B_2_CDR3nt']
seq_set_ABnt = generate_clone_sets(masked_ABnt,cols)
tcr_ABnt = concat_seqs_in_set(seq_set_ABnt)

In [53]:
cols = ['A_1_CDR3aa','A_2_CDR3aa','B_1_CDR3aa','B_2_CDR3aa']
seq_set_ABaa = generate_clone_sets(masked_ABaa,cols)
tcr_ABaa = concat_seqs_in_set(seq_set_ABaa)

In [54]:
cols = ['G_1_CDR3nt','G_2_CDR3nt','D_1_CDR3nt','D_2_CDR3nt']
seq_set_GDnt = generate_clone_sets(masked_GDnt,cols)
tcr_GDnt = concat_seqs_in_set(seq_set_GDnt)

In [55]:
cols = ['G_1_CDR3aa','G_2_CDR3aa','D_1_CDR3aa','D_2_CDR3aa']
seq_set_GDaa = generate_clone_sets(masked_GDaa,cols)
tcr_GDaa = concat_seqs_in_set(seq_set_GDaa)

## 5.2 Group clones + frequency

In [57]:
# ABnt
df.insert(len(df.columns),'TCR_AB_nt',tcr_ABnt)
df = group_with_freq(df,'TCR_AB_nt',group_unique=False,new_name='clone_ABnt')
# ABaa
df.insert(len(df.columns),'TCR_AB_aa',tcr_ABaa)
df = group_with_freq(df,'TCR_AB_aa',group_unique=False,new_name='clone_ABaa')
# GDnt
df.insert(len(df.columns),'TCR_GD_nt',tcr_GDnt)
df = group_with_freq(df,'TCR_GD_nt',group_unique=False,new_name='clone_GDnt')
# GDaa
df.insert(len(df.columns),'TCR_GD_aa',tcr_GDaa)
df = group_with_freq(df,'TCR_GD_aa',group_unique=False,new_name='clone_GDaa')

In [58]:
df

,Plate,A_1_productive,A_1_TPM,A_1_stop_codon,A_1_in_frame,A_1_ID,A_1_CDR3nt,A_1_CDR3aa,A_1_V,A_1_J,A_2_productive,A_2_TPM,A_2_stop_codon,A_2_in_frame,A_2_ID,A_2_CDR3nt,A_2_CDR3aa,A_2_V,A_2_J,A_productive,B_1_productive,B_1_TPM,B_1_stop_codon,B_1_in_frame,B_1_ID,B_1_CDR3nt,B_1_CDR3aa,B_1_V,B_1_J,B_1_D,B_2_productive,B_2_TPM,B_2_stop_codon,B_2_in_frame,B_2_ID,B_2_CDR3nt,B_2_CDR3aa,B_2_V,B_2_J,B_2_D,B_productive,AB_productive,G_1_productive,G_1_TPM,G_1_stop_codon,G_1_in_frame,G_1_ID,G_1_CDR3nt,G_1_CDR3aa,G_1_V,G_1_J,G_2_productive,G_2_TPM,G_2_stop_codon,G_2_in_frame,G_2_ID,G_2_CDR3nt,G_2_CDR3aa,G_2_V,G_2_J,G_productive,D_1_productive,D_1_TPM,D_1_stop_codon,D_1_in_frame,D_1_ID,D_1_CDR3nt,D_1_CDR3aa,D_1_V,D_1_J,D_1_D,D_2_productive,D_2_TPM,D_2_stop_codon,D_2_in_frame,D_2_ID,D_2_CDR3nt,D_2_CDR3aa,D_2_V,D_2_J,D_2_D,D_productive,GD_productive,TCR_AB_nt,freq_clone_ABnt,group_clone_ABnt,TCR_AB_aa,freq_clone_ABaa,group_clone_ABaa,TCR_GD_nt,freq_clone_GDnt,group_clone_GDnt,TCR_GD_aa,freq_clone_GDaa,group_clone_GDaa
Plate231_PB_L14,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,,153,0,,153,0,,289,0,,289,0
Plate231_PB_P17,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,,153,0,,153,0,,289,0,,289,0
Plate231_MUSL_E16,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,,153,0,,153,0,,289,0,,289,0
Plate231_PB_M21,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,,153,0,,153,0,,289,0,,289,0
Plate231_MUSL_F15,SS3_21_231-1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,,153,0,,153,0,,289,0,,289,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate231_PB_I4,SS3_21_231-3,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,1,463.311,0,1,TRBV28_GCAGTCGCCTCCGGGACAGGATCGACTTC_TRBJ2-7,GCCAGCAGTCGCCTCCGGGACAGGATCGAC,ASSRLRDRID,TRBV28*01,TRBJ2-7*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,GCCAGCAGTCGCCTCCGGGACAGGATCGAC,1,72,ASRNGQGGAFARNNAGNMLT,1,108,,289,0,ACDTRVYWGIRCTDKLI,1,7
Plate231_MUSL_C11,SS3_21_231-3,1,1022.59,0,1,TRAV12-1_GAACATAGGGGG_TRAJ13,GTGGTGAACATAGGGGGTTACCAGAAAGTTACC,VVNIGGYQKVT,TRAV12-1*01,TRAJ13*02,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,1,795.899,0,1,TRBV12-3_GCAGTCCCGGGACAGGGAATGACTAT_TRBJ1-2,GCCAGCAGTCCCGGGACAGGGAATGACTATGGCTACACC,ASSPGTGNDYGYT,TRBV12-3*01,TRBJ1-2*01,TRBD1*01,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,N

In [60]:
with pd.option_context('display.max_rows',None):
    display(df.loc[:,['TCR_AB_aa','freq_clone_ABaa','group_clone_ABaa']].\
            sort_values('group_clone_ABaa',ascending=True))

,TCR_AB_aa,freq_clone_ABaa,group_clone_ABaa
Plate231_PB_L14,,153,0
Plate231_MUSL_A17,,153,0
Plate231_MUSL_G4,,153,0
Plate231_MUSL_H9,,153,0
Plate231_PB_N2,,153,0
Plate231_PB_N10,,153,0
Plate231_MUSL_H21,,153,0
Plate231_MUSL_C13,,153,0
Plate231_PB_N21,,153,0
Plate231_PB_N19,,153,0


## 6. Export dataset

In [ ]:
df.to_tsv(out_file,sep='\t')